In [17]:
# Cargar bases de datos de carpeta data
import pandas as pd

# Base de datos
df1 = pd.read_csv("data/BD_Asegurados_Expuestos.csv", sep=";")
df2 = pd.read_csv("data/BD_Diagnostico.csv", sep=";")
df3 = pd.read_csv("data/BD_Genero.csv", sep=";")
df4 = pd.read_csv("data/BD_Reclamaciones.csv", sep=";")
df5 = pd.read_csv("data/BD_Regional.csv", sep=";")
df6 = pd.read_csv("data/BD_SocioDemograficas.csv", sep=";", low_memory=False)
df7 = pd.read_csv("data/BD_UtilizacionesMedicas.csv", sep=";")
fechas = pd.read_csv("data/fechas.csv", sep=";")

# Unión bases de datos

Procesamiento de fechas
1. Se utiliza la funcion UNICOS en cada libro de excel
2. Se pasan esos valores a un libro de excel llamado fechas en una sola columna
4. Se vuelve a realizar la función UNICOS para obtener los unicos de todos los libros
5. Se crea columna con fecha
6. Se realiza unión de bases de datos

In [18]:
################################ ASIGNACIÓN DE FECHAS CON BASE FECHAS ##################################
# Fecha de inicio, cancelación y fin
df1 = (
    pd.merge(df1, fechas, left_on="FECHA_INICIO", right_on="cod_fecha", how="left")
    .merge(fechas, left_on="FECHA_CANCELACION", right_on="cod_fecha", how="left")
    .merge(fechas, left_on="FECHA_FIN", right_on="cod_fecha", how="left")
    .drop(
        columns=[
            "cod_fecha",
            "FECHA_INICIO",
            "FECHA_FIN",
            "FECHA_CANCELACION",
            "cod_fecha",
            "cod_fecha_x",
            "cod_fecha_y",
        ]
    )
    .rename(
        columns={
            "fecha_x": "FECHA_INICIO",
            "fecha_y": "FECHA_CANCELACION",
            "fecha": "FECHA_FIN",
        }
    )
)

# Fecha de nacimiento
df6 = (
    pd.merge(df6, fechas, left_on="FechaNacimiento", right_on="cod_fecha", how="left")
    .drop(columns=["cod_fecha", "FechaNacimiento"])
    .rename(columns={"fecha": "FechaNacimiento"})
)

# Fecha de reclamación
df7 = (
    pd.merge(df7, fechas, left_on="Fecha_Reclamacion", right_on="cod_fecha", how="left")
    .drop(columns=["cod_fecha", "Fecha_Reclamacion"])
    .rename(columns={"fecha": "Fecha_Reclamacion"})
)

###############################  Convertir a datetime las columnas de fechas ###############################
df1["FECHA_INICIO"] = pd.to_datetime(df1["FECHA_INICIO"], format="%d/%m/%Y")
df1["FECHA_CANCELACION"] = pd.to_datetime(df1["FECHA_CANCELACION"], format="%d/%m/%Y")
df1["FECHA_FIN"] = pd.to_datetime(df1["FECHA_FIN"], format="%d/%m/%Y")
df6["FechaNacimiento"] = pd.to_datetime(df6["FechaNacimiento"], format="%d/%m/%Y")
df7["Fecha_Reclamacion"] = pd.to_datetime(df7["Fecha_Reclamacion"], format="%d/%m/%Y")

In [19]:
######################################### UNION DE BASES DE DATOS #########################################

######################## df7 = Union de bases Diagnostico y reclamación ##############################

df7 = pd.merge(df7, df2, on="Diagnostico_Codigo", how="left").merge(
    df4, on="Reclamacion_Cd", how="left"
)

####################### Procesamiento para merge Regional_Id ##############################
# Cambiar nombre de Regional a Regional_Id de df6
df6 = df6.rename(columns={"Regional": "Regional_Id"})

# Reemplazar de Regional_Id de df6 el #N/D por -1
df6["Regional_Id"] = df6["Regional_Id"].replace("#N/D", -1)

# Convertir Regional_Id de df6 a object
df6["Regional_Id"] = df6["Regional_Id"].astype("int64")

####################### df6 = Union de bases Sexo, Regional ##############################
df6 = pd.merge(df6, df3, on="Sexo_Cd", how="left").merge(
    df5, on="Regional_Id", how="left"
)


####################### Unión final ##############################
df1 = df1.rename(columns={"Asegurado_Id": "Afiliado_Id"})

# Union de df1, df6 y df7 con Asegurado_Id
df = pd.merge(df7, df6, on="Afiliado_Id", how="outer").merge(
    df1, on="Afiliado_Id", how="outer"
)

df.shape

(894056, 22)

# Limpieza de datos

In [20]:
######################################### TRATAMIENTOS MENORES #########################################
# Cambiar Sexo_Cd.1 por Sexo
df = df.rename(columns={"Sexo_Cd.1": "Sexo"})

# cambiar nombre columna Valor_Utilizaciones por Costo_Total_Utilizaciones
df = df.rename(columns={"Valor_Utilizaciones": "Costo_Total_Utilizaciones"})


######################################### LIMPIEZA #########################################
# quitar categoría "Sin información" de Sexo (5 datos)
df = df[df["Sexo"] != "Sin Informacion"]

# Quitar categoría "Sin información" de Regional_desc (acumulado 110)
df = df[df["Regional_desc"] != "Sin Información"]

# Seleccionar solo los registros del 2019 (acumulado 70599)
df = df[df["Fecha_Reclamacion"].dt.year == 2019]


######################################### CREACIÓN DE COLUMNAS #########################################
# Crear columna con FechaNacimiento para edad
df["Edad"] = df["Fecha_Reclamacion"].dt.year - df["FechaNacimiento"].dt.year
# Se eliminan las filas con Edad negativa (acumulado 73236)
df = df[df["Edad"] >= 0]


# Columna con Valor_Utilizaciones / Numero_Utilizaciones
df["Precio_Utilización"] = df["Costo_Total_Utilizaciones"] / df["Numero_Utilizaciones"]

# Cambiar nombre de columnas Numero_Utilizaciones por Cantidad_Utilizaciones y Valor_Utilizaciones por Costo_Total_Utilización
df = df.rename(
    columns={
        "Numero_Utilizaciones": "Cantidad_Utilizaciones",
    }
)

# Duración póliza = FECHA_FIN - FECHA_INICIO en días redondeando a 0 decimales
df["Duración_Póliza"] = (((df["FECHA_FIN"] - df["FECHA_INICIO"]).dt.days) / 30).round()

# Columna por rangos de "Duración_Póliza" así: si es 0 = "<= 0.5 Meses", 1 = "0.5-1.5 Meses", 2 = "1.5-2.5 Meses", 3 = "2.5-3.5 Meses", 4 = ">= 3.5 Meses"
df["Duración_Póliza_Rango"] = pd.cut(
    df["Duración_Póliza"],
    bins=[-1, 0.5, 1.5, 2.5, 3.5, 100],
    labels=[
        "<= 0.5 Meses",
        "0.5-1.5 Meses",
        "1.5-2.5 Meses",
        "2.5-3.5 Meses",
        ">= 3.5 Meses",
    ],
)

# Días_en_reclamar = FECHA_FIN - FECHA_INICIO en días redondeando a 0 decimales y quitando los valores negativos
df["Días_en_reclamar"] = (
    ((df["Fecha_Reclamacion"] - df["FECHA_INICIO"]).dt.days)
).round()


# Acá se quitan los valores negativos, no tiene sentido que se reclame antes de que inicie la póliza (acumulado 255989)
df = df[df["Días_en_reclamar"] >= 0]

################################## Columnas a eliminar ##################################

df = df.drop(
    columns=[
        "Reclamacion_Cd",  # Se tomó el valor y no el código
        "Diagnostico_Codigo",  # Se tomó el valor y no el código
        "Sexo_Cd",  # Se tomó el valor y no el código
        "Regional_Id",  # Se tomó el valor y no el código
        "FechaNacimiento",  # Se tomó la edad
        "FECHA_CANCELACION",  # Se tomó la fecha de fin
        "Duración_Póliza",  # Se tomó el rango
    ]
)

df.shape

(638024, 20)

TRATAMIENTOS MENORES
* Cambiar Sexo_Cd.1 por Sexo
* Cambiar nombre columna Valor_Utilizaciones por Costo_Total_Utilizaciones

LIMPIEZA DE DATOS
* Quitamos la categoría "Sin Información" de la variable Regional_desc (110 datos) y de la variable Sexo (5 datos)
* Nos centraremos en los afiliados que compraron la póliza en septiembre de 2019 y la reclamaron después de esa fecha. Además tomamos esta decisión porque el número de afiliados restante es muy bajo del 10%, con lo que tendríamos el 90% para realizar el análisis.
* Se quitaron las edades menores a 0
* Los días a reclamar menores a 0 se eliminaron porque nos enfocamos en los que reclamaron después de la compra de la póliza

CREACIÓN DE COLUMNAS
* Con la columna fecha de reclamación y fecha de nacimiento se crea una columna con la edad
* Creación columna precio unitario = Costo_Total_Utilizaciones / Numero_Utilizaciones
* Duración póliza = FECHA_FIN - FECHA_INICIO
* Se crea una columna con el rango de duración de la póliza
* Días_en_reclamar = FECHA_FIN - FECHA_INICIO


ELIMINACIÓN DE COLUMNAS

Porque se tomó el valor y no el código
* Reclamacion_Cd 
* Diagnostico_Codigo
* Sexo_Cd
* Regional_Id


Porque se tomó otra columna 
* FechaNacimiento: Se tomó la edad
* FECHA_CANCELACION: Era la misma de FECHA_FIN
* Duración_Póliza: Se tomó el rango

Utilizaremos los Modelos individuales de utilizaciones en vez de las sumas de utilizaciones porque ofrecen las siguientes ventajas:

* Mayor detalle: Modelar las utilizaciones por separado (por ejemplo, visitas médicas, procedimientos, exámenes) proporciona un mayor nivel de detalle y pueden ayudar a comprender mejor qué tipos de servicios médicos se están utilizando y en qué cantidad.
* Personalización: Permite una personalización más precisa de las predicciones, ya que diferentes pacientes pueden utilizar diferentes tipos y cantidades de servicios médicos.

Sin embargo, en el análisis exploratorio de datos, utilizaremos las sumas de utilizaciones y costos totales por paciente para tener una idea general de la cantidad de servicios médicos utilizados y su costo total.

In [23]:
# conversión de variables 'Afiliado_Id' y 'EPOC', 'DIABETES','HIPERTENSION','ENF_CARDIOVASCULAR,'Poliza_Id, 'CANCER'
# en variables categóricas
for col in [
    "Afiliado_Id",
    "EPOC",
    "DIABETES",
    "HIPERTENSION",
    "ENF_CARDIOVASCULAR",
    "Poliza_Id",
    "CANCER",
]:
    df[col] = df[col].astype("object")


################################## cambiar orden de las columnas ##################################

df = df[
    [
        "Afiliado_Id",
        "Sexo",
        "Edad",
        "Regional_desc",
        "Poliza_Id",
        "Fecha_Reclamacion",
        "FECHA_INICIO",
        "FECHA_FIN",
        "Días_en_reclamar",
        "Cantidad_Utilizaciones",
        "Precio_Utilización",
        "Costo_Total_Utilizaciones",
        "Diagnostico_Desc",
        "Reclamacion_Desc",
        "CANCER",
        "EPOC",
        "DIABETES",
        "HIPERTENSION",
        "ENF_CARDIOVASCULAR",
        "Duración_Póliza_Rango",
    ]
]

## Tratamiento datos nulos

In [24]:
# Nulos de df porcentaje
print(df.shape)
print(df.isnull().sum() / len(df) * 100)

(638024, 20)
Afiliado_Id                  0.0
Sexo                         0.0
Edad                         0.0
Regional_desc                0.0
Poliza_Id                    0.0
Fecha_Reclamacion            0.0
FECHA_INICIO                 0.0
FECHA_FIN                    0.0
Días_en_reclamar             0.0
Cantidad_Utilizaciones       0.0
Precio_Utilización           0.0
Costo_Total_Utilizaciones    0.0
Diagnostico_Desc             0.0
Reclamacion_Desc             0.0
CANCER                       0.0
EPOC                         0.0
DIABETES                     0.0
HIPERTENSION                 0.0
ENF_CARDIOVASCULAR           0.0
Duración_Póliza_Rango        0.0
dtype: float64


# Guardar CSV

In [25]:
# Guardar base de datos
df.to_csv("data/data_clean.csv", index=False)